In [1]:
from glob import glob
# import sys
from dotenv import load_dotenv

load_dotenv()
# sys.path.append("/u/zliu/datastor1/KE-by-CP")
import pandas as pd
# from experiments.musique.inference_only import macro_averaging
from knowledge_propagation.utils import io, vars, extractor
from collections import Counter
import os
import numpy as np
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import describe
from thefuzz import fuzz

from datasets import load_dataset, load_from_disk

from copy import deepcopy

from dateutil.parser import parse
from dateutil.parser import ParserError

from collections import defaultdict

import re

def is_date(string):
    try:
        parse(string)
        return True
    except ParserError:
        return False

### Bio data with more template (v2)

In [596]:
all_country_questions_prefilter = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/all_country_questions_prefilter.jsonl")
# all_country_questions_prefilter = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/world_knowledge/country_knowledge_questions.jsonl.bak")
country2continent = io.load_json(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/data_gen/country2continent.json")
len(all_country_questions_prefilter)

1185

In [597]:
# # fix template
# fixed_ones = []
# for x in all_country_questions_prefilter:
#     new_x = deepcopy(x)
#     if new_x["template"] == "Which country is the largest in {continent} by area?":
#         # break
#         new_x["template"] = "Which country is the largest by area in {continent}?"
#         new_x["question"] = new_x["template"].format(continent=country2continent[new_x["key"]])
#         # fixed_ones.append(new_x)
#     fixed_ones.append(new_x)
# all_country_questions_prefilter = fixed_ones

In [598]:
# io.dump_jsonlines(all_country_questions_prefilter, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/world_knowledge/country_knowledge_questions.jsonl")

In [599]:
city_country_pairs = pd.DataFrame(all_country_questions_prefilter)["(city, country)"].dropna().tolist()

city2country = {}
for pair in city_country_pairs:
    city, country = pair
    if city not in city2country:
        city2country[city] = []
    city2country[city].append(country)

In [600]:
# city only belongs to one country
[(k,v) for k, v in city2country.items() if len(v) > 1]

[]

In [601]:
country2cities = defaultdict(list)
for city_country_pair in city_country_pairs:
    city, country = city_country_pair
    country2cities[country].append(city)
# delete French Polynesia since the performance is low
del country2cities["French Polynesia"]
len(country2cities)

75

In [602]:
city2country = dict(city_country_pairs)
assert len(city_country_pairs) == len(city2country)
all([c in country2continent for c in city2country.values()])

True

In [603]:
countries_by_city_count = [x[0] for x in sorted(country2cities.items(), key=lambda x: len(x[1]), reverse=True)]

In [575]:
high_countries = countries_by_city_count[:len(countries_by_city_count)//2]
low_countries = countries_by_city_count[len(countries_by_city_count)//2:]

In [351]:
def sample_countries_by_city_count(country_cities_dict, n, proportional=True, weighted=True):
    
    """randomly sample n countries so that the probability to sample that country is proportional to the number of cities in that country

    Args:
        country_cities_dict (_type_): _description_
        n (_type_): _description_

    Returns:
        _type_: _description_
    """
    # Calculate weights as the number of cities in each country
    countries = list(country_cities_dict.keys())
    if weighted:
        if proportional:
            weights = [len(cities) for cities in country_cities_dict.values()]
        else:
            weights = [1 / (len(cities) + 0.1) for cities in country_cities_dict.values()]
    else:
        weights = [1] * len(countries)
    weights = np.array(weights) / np.sum(weights)
    
    # Use random.choices which allows weighted sampling with replacement
    sampled_countries = np.random.choice(
        countries,
        size=n,
        replace=False,
        p=weights,
    )
    
    return sampled_countries.tolist()

In [604]:
# unseen_high_countries = sample_countries_by_city_count({k: country2cities[k] for k in high_countries}, 7, weighted=False)
# unseen_low_countries = sample_countries_by_city_count({k: country2cities[k] for k in low_countries}, 7, weighted=False)
# unseen_countries = unseen_high_countries + unseen_low_countries
# unseen_countries

In [606]:
# io.dump_jsonlines(unseen_countries, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/unseen_countries.jsonl")
unseen_countries = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/data_gen/unseen_countries.jsonl")
unseen_countries

['Czech Republic',
 'Thailand',
 'Belgium',
 'Finland',
 'Egypt',
 'Singapore',
 'Turkey',
 'Congo',
 'Italy',
 'Colombia',
 'Nigeria',
 'Iceland',
 'Bolivia',
 'Saudi Arabia']

In [657]:
place_template2template_tag = io.load_json(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/data_gen/place_template2template_tag.json")
place_template2template_paraphrases = io.load_json(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/data_gen/place_template2template_paraphrases.json")


In [508]:
# io.dump_json(place_template2template_tag, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/place_template2template_tag.json")
# io.dump_json(place_template2template_paraphrases, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/place_template2template_paraphrases.json")

In [509]:
# acc_by_country_df[~acc_by_country_df["key"].isin(unseen_countries)]
# sns.barplot(data=acc_by_country_df, x="mean", y="key", orient="h", errorbar=None,)
# sns.barplot(data=df, x="llm_accuracy", y="template_tag",  orient="h",)

In [607]:
country2cities_sc = {country: cities for country, cities in country2cities.items() if country not in unseen_countries} #
city_country_pairs_sc = []
for country, cities in country2cities_sc.items():
    for city in cities:
        city_country_pairs_sc.append((city, country))
print("# seen (city, country) pairs [from seen country]:", len(city_country_pairs_sc))
np.random.shuffle(city_country_pairs_sc)

# seen (city, country) pairs [from seen country]: 449


In [688]:
# n_seen_pairs = 350
# seen_city_country_pairs_sc = city_country_pairs_sc[:n_seen_pairs]
# unseen_city_country_pairs_sc = city_country_pairs_sc[n_seen_pairs:]
# io.dump_jsonlines(seen_city_country_pairs_sc, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/data_gen/seen_city_country_pairs_from_seen_countries.jsonl")
# io.dump_jsonlines(unseen_city_country_pairs_sc, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/data_gen/unseen_city_country_pairs_from_seen_countries.jsonl")
seen_city_country_pairs_sc = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/data_gen/seen_city_country_pairs_from_seen_countries.jsonl")
unseen_city_country_pairs_sc = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/data_gen/unseen_city_country_pairs_from_seen_countries.jsonl")

In [609]:
country2cities_uc = {country: cities for country, cities in country2cities.items() if country in unseen_countries}

city_country_pairs_uc = []
for country, cities in country2cities_uc.items():
    for city in cities:
        city_country_pairs_uc.append((city, country))
print("# unseen (city, country) pairs:", len(city_country_pairs_uc))

# unseen (city, country) pairs: 122


In [684]:
# io.dump_jsonlines(city_country_pairs_uc, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/data_gen/unseen_city_country_pairs_from_unseen_countries.jsonl")
city_country_pairs_uc = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/data_gen/unseen_city_country_pairs_from_unseen_countries.jsonl")

In [611]:
all([t in place_template2template_tag for t in set(x["template"] for x in all_country_questions_prefilter)])

True

In [613]:
# basic_templates = ['Which country is {city} located in?', 'Which continent is {country} located in?',] 
# advanced_templates = [k for k in place_template2template_tag.keys() if k not in basic_templates]
# np.random.shuffle(advanced_templates)
# all_tempaltes_reordered = basic_templates + advanced_templates

In [668]:
first_names = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/data_gen/first_names.jsonl")
last_names = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/data_gen/last_names.jsonl")
careers = [c.lower() for c in io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/data_gen/careers.jsonl")]
gender = ["he", "she"]

In [669]:
# io.dump_jsonlines(all_tempaltes_reordered, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/all_tempaltes_reordered.jsonl")
all_tempaltes_reordered = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/data_gen/all_tempaltes_reordered.jsonl")

In [670]:
all_tempaltes_reordered

['Which country is {city} located in?',
 'Which continent is {country} located in?',
 'What language has the most speakers in {country}?',
 'Which religion has the most followers in {country}?',
 'What is the full official name of {country}?',
 'Which ethnic group is the largest in {country}?',
 'What is the capital of {country}?',
 'What is the currency of {country}?',
 'Which country is the largest by area in {continent}?']

In [671]:
# io.load_jsonlines("/u/zliu/datastor1/KE-by-CP/data/debug_meta_train/country_syn_data_v2/unseen_countries.jsonl")

In [672]:
text_template = "{first_name} {last_name} was born in {birth_place}. {gender_start} started and completed a career as {career} in {career_place}. After retirement, {gender} lived in {death_place} and passed away."


# Example function to extract information
def extract_info_country(text):
    pattern = r"(?P<first_name>\w+) (?P<last_name>\w+) was born in (?P<birth_place>[\w\s,'.()-]+)\. (?P<gender_start>\w+) started and completed a career as (?P<career>[\w\s]+) in (?P<career_place>[\w\s,'.()-]+)\. After retirement, (?P<gender>\w+) moved to (?P<death_place>[\w\s,'.()-]+) and passed away\."
    match = re.search(pattern, text)
    if match:
        ret = match.groupdict()
        assert "career" in ret
        ret["career"] = ret["career"].split(" ")[1:] # remove the article
        return ret
    return None


In [673]:
def knowledge_triplets_from_text(text):
    info = extract_info_country(text)
    
    full_name = f"{info['first_name']} {info['last_name']}"
    birth_place = info["birth_place"]
    career = info["career"]
    career_place = info["career_place"]
    death_place = info["death_place"]
    gender = info["gender"]
    g = "male" if gender == "he" else "female"
    gender_pronoun = "his" if gender == "he" else "her"
        
    return [
        {
            "subject": full_name,
            "prompt": f"{full_name} was born in",
            "answer": birth_place,
            "relation": "birth_place",
            "paraphrase_prompts": [
                f"{full_name} hailed from",
                f"Which city was {full_name} born in?",
                f"{full_name} was originally from",
                f"{full_name} came into the world in",
            ]
        },
        {
            "subject": full_name,
            "prompt": f"{full_name} worked in the field of",
            "answer": career,
            "relation": "career",
            "paraphrase_prompts": [
                f"{full_name} dedicated {gender_pronoun} professional life to",
                f"What did {full_name} do for a living?",
                f"{full_name} was a",
                f"{full_name} worked as a",
            ]
        },
        {
            "subject": full_name,
            "prompt": f"{full_name} worked in",
            "answer": career_place,
            "relation": "career_place",
            "paraphrase_prompts": [
                f"Where did {full_name} work?",
                f"{full_name} dedicated {gender_pronoun} professional life to",
                f"{full_name} developed {gender_pronoun} career while living in",
                f"Where did {full_name} develop {gender_pronoun} career?",
            ]
                
        },
        {
            "subject": full_name,
            "prompt": f"{full_name} passed away in",
            "answer": death_place,
            "relation": "death_place",
            "paraphrase_prompts": [
                f"Where did {full_name} pass away?",
                f"{full_name} died in",
                f"{full_name} deceased in",
                f"Where did {full_name} die?",
            ]
            
        },
        {
            "subject": full_name,
            "prompt": f"{full_name} was a",
            "answer": g,
            "relation": "gender",
            "paraphrase_prompts": [
                f"{full_name} was identified as",
                f"{full_name} was recognized as",
                f"What was the gender of {full_name}?",
                f"Which gender does {full_name} belong to?",
            ]
        },
    ]
    

# Generate place meta-training data

### meta-training data for country

In [674]:

country_events = [
    "was {first_name} {last_name} born in",
    "did {first_name} {last_name} work in",
    "did {first_name} {last_name} die in",
]

country_events_in_statement = [
    "{first_name} {last_name} was born in",
    "{first_name} {last_name} worked in",
    "{first_name} {last_name} died in",
]
country_templates = [
    ("Which country" + " " + country_events[0] + "?", lambda *c: city2country[c[0]]),
    ("Which country" + " " + country_events[1] + "?", lambda *c: city2country[c[1]]),
    ("Which country" + " " + country_events[2] + "?", lambda *c: city2country[c[2]]),
]
continent_templates = [
    ("Which continent" + " " + country_events[0] + "?", lambda *c: country2continent[city2country[c[0]]]),
    ("Which continent" + " " + country_events[1] + "?", lambda *c: country2continent[city2country[c[1]]]),
    ("Which continent" + " " + country_events[2] + "?", lambda *c: country2continent[city2country[c[2]]]),
]

In [675]:
len(all_country_questions_prefilter)

1185

In [676]:
import inflect

p = inflect.engine()
word = "elephant"
article = p.a(word) 
article

'an elephant'

In [681]:
question2answer = {}
dup_questions = []
for x in all_country_questions_prefilter:
    if x["question"] not in question2answer:
        question2answer[x["question"]] = x["answer"]
    else:
        # there's a unique answer for each question
        dup_questions.append(x["question"])
        assert question2answer[x["question"]] == x["answer"], f"Q: {x['question']} A: {question2answer[x['question']]} vs {x['answer']}"
print("# questions:", len(question2answer))
print("# entries:", len(all_country_questions_prefilter))
assert all(["Which country is the largest by" in x for x in dup_questions])

# questions: 1114
# entries: 1185


In [678]:
question2answer["Which country is the largest by area in Asia?"]

'Russia'

In [ ]:
def knowledge_propagation_syn_data_from_pairs(pairs, gen_templates, n_data=100000, loc_nochange_prob=0.2):
    syn_data = []
    tuples = set()

    # pbar = tqdm(total = n_data)

    no_ans_questions = []

    while len(syn_data) < n_data:
        
        first_name = np.random.choice(first_names)
        last_name = np.random.choice(last_names)

        gender = np.random.choice(genders)
        career = np.random.choice(careers)
        
        birth_cc = pairs[np.random.choice(len(pairs))]
        if np.random.rand() < loc_nochange_prob:
            career_cc = birth_cc
        else:
            career_cc = pairs[np.random.choice(len(pairs))]
        
        if np.random.rand() < loc_nochange_prob:
            death_cc = career_cc
        else:
            death_cc = pairs[np.random.choice(len(pairs))]
        
        birth_ccc = (*birth_cc, country2continent[birth_cc[1]])
        career_ccc = (*career_cc, country2continent[career_cc[1]])
        death_ccc = (*death_cc, country2continent[death_cc[1]])
        
        info_tuple = (first_name, last_name, birth_ccc[0], career_ccc[0], gender, career, death_ccc[0])
        
        if info_tuple not in tuples:
            tuples.add(info_tuple)
        else:
            continue

        text = text_template.format(
            first_name=first_name,
            last_name=last_name,
            birth_place=birth_ccc[0],
            gender=gender,
            gender_start=gender.capitalize(),
            career=p.a(career),
            career_place=career_ccc[0],
            death_place=death_ccc[0],
        )
        
        questions = []
        for unalias_template in gen_templates:
            paraphrase_question_template = np.random.choice(place_template2template_paraphrases[unalias_template])
            
            if unalias_template == "Which country is {city} located in?":
                rand_idx = np.random.choice(range(len(country_templates)))
                
                question_template, _ = country_templates[rand_idx]
                question = question_template.format(
                    first_name=first_name,
                    last_name=last_name,
                    career=career,
                    gender=gender
                )
                
                # answer_fn is buggy
                answer = [birth_ccc[1], career_ccc[1], death_ccc[1]][rand_idx]
                
                unalias_question = unalias_template.format(city=[birth_ccc[0], career_ccc[0], death_ccc[0]][rand_idx])
                assert unalias_question in question2answer
                
                assert answer == question2answer[unalias_question], f"Q: {unalias_question} A: {answer} vs {question2answer[unalias_question]}"

                city_statement = f"the city that {country_events_in_statement[rand_idx].format(first_name=first_name, last_name=last_name)}"
                paraphrase_question = paraphrase_question_template.format(city=city_statement)
                unalias_paraphrase_question = paraphrase_question_template.format(city=[birth_ccc[0], career_ccc[0], death_ccc[0]][rand_idx])
                
            elif unalias_template == "Which continent is {country} located in?":
                rand_idx = np.random.choice(range(len(continent_templates)))
                
                question_template, answer_fn = continent_templates[rand_idx]
                question = question_template.format(
                    first_name=first_name,
                    last_name=last_name,
                    career=career,
                    gender=gender
                )
                answer = [birth_ccc[2], career_ccc[2], death_ccc[2]][rand_idx]
                
                unalias_question = unalias_template.format(country=[birth_ccc[1], career_ccc[1], death_ccc[1]][rand_idx])
                assert unalias_question in question2answer
                assert answer == question2answer[unalias_question], f"Q: {unalias_question} A: {answer} vs {question2answer[unalias_question]}"

                country_statement = f"the country that {country_events_in_statement[rand_idx].format(first_name=first_name, last_name=last_name)}"
                paraphrase_question = paraphrase_question_template.format(country=country_statement)
                unalias_paraphrase_question = paraphrase_question_template.format(country=[birth_ccc[1], career_ccc[1], death_ccc[1]][rand_idx])
                
            else:
                
                templates = country_events_in_statement
                rand_idx = np.random.choice(range(len(templates)))
                
                country_statement = f"the country that {templates[rand_idx].format(first_name=first_name, last_name=last_name)}"
                chosen_country = [birth_ccc[1], career_ccc[1], death_ccc[1]][rand_idx]
                
                continent_statement = f"the continent that {templates[rand_idx].format(first_name=first_name, last_name=last_name)}"
                chosen_continent = [birth_ccc[2], career_ccc[2], death_ccc[2]][rand_idx]
                
                unalias_question = unalias_template.format(country=chosen_country, continent=chosen_continent)
                question = unalias_template.format(country=country_statement, continent=continent_statement)
                if unalias_question not in question2answer:
                    no_ans_questions.append(unalias_question)
                    continue
                # assert unalias_question in question2answer, unalias_question
                answer = question2answer[unalias_question]
                paraphrase_question = paraphrase_question_template.format(country=country_statement, continent=continent_statement)
                unalias_paraphrase_question = paraphrase_question_template.format(country=chosen_country, continent=chosen_continent)
                
            questions.append(
                {
                    "template": unalias_template,
                    "template_tag": place_template2template_tag[unalias_template],
                    "question": question,
                    "paraphrase_question": paraphrase_question,
                    "unalias_question": unalias_question,
                    "unalias_paraphrase_question": unalias_paraphrase_question,
                    "answer": answer,
                    
                }
            )
        syn_data.append(
            {
                "text": text,
                "questions": questions
            }
        )
        # pbar.update(1)
    return syn_data

In [710]:
loc_nochange_prob = 0.2
n_data = 100_000
n_test = 100
n_dev = 100
n_train = n_data - n_test - n_dev

ood_pairs_sc = unseen_city_country_pairs_sc
ood_pairs_uc = city_country_pairs_uc

id_country_ood_pair_test = knowledge_propagation_syn_data_from_pairs(ood_pairs_sc, all_tempaltes_reordered, n_data=n_test, loc_nochange_prob=loc_nochange_prob)
ood_country_test = knowledge_propagation_syn_data_from_pairs(ood_pairs_uc, all_tempaltes_reordered, n_data=n_test, loc_nochange_prob=loc_nochange_prob)

In [714]:
io.dump_jsonlines(id_country_ood_pair_test, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/test_unseen_pairs_seen_country.jsonl")
io.dump_jsonlines(ood_country_test, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/test_unseen_pairs_unseen_country.jsonl")

In [713]:

# n_seen_pairs=50


for n_template in tqdm([1, 7]):
    for n_seen_pairs in tqdm([100, 200,]): # [50, 100, 200, 350]
        save_dir = "n_template_{}_n_seen_pairs_{}".format(n_template, n_seen_pairs)
        save_dir_fullpath = f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/{save_dir}"
        os.makedirs(save_dir_fullpath, exist_ok=True)


        id_templates = all_tempaltes_reordered[:n_template]
        ood_templates = all_tempaltes_reordered[n_template:]

        id_pairs_sc = seen_city_country_pairs_sc[:n_seen_pairs]

        syn_data = knowledge_propagation_syn_data_from_pairs(id_pairs_sc, all_tempaltes_reordered, n_data=n_data, loc_nochange_prob=loc_nochange_prob)
        rand_shuffle = np.arange(n_data)
        np.random.shuffle(rand_shuffle)

        train_data = [syn_data[i] for i in rand_shuffle[:n_train]]
        dev_data = [syn_data[i] for i in rand_shuffle[n_train:n_train+n_dev]]
        test_data = [syn_data[i] for i in rand_shuffle[n_train+n_dev:]]

        io.dump_jsonlines(
            [
                {
                    "text": x["text"],
                    "questions": [q for q in x["questions"] if q["template"] in id_templates],
                }
                for x in train_data
            ], f"{save_dir_fullpath}/train.jsonl")
        io.dump_jsonlines([
                {
                    "text": x["text"],
                    "questions": [q for q in x["questions"] if q["template"] in id_templates],
                }
                for x in dev_data
            ], f"{save_dir_fullpath}/valid.jsonl")
        io.dump_jsonlines(
            [
                {
                    "text": x["text"],
                    "questions": [q for q in x["questions"] if q["template"] in id_templates],
                    "ood_questions": [q for q in x["questions"] if q["template"] not in id_templates],
                }
                for x in test_data
            ], f"{save_dir_fullpath}/test_seen_pairs_seen_country.jsonl")

        
        io.dump_jsonlines([
            {
                "text": x["text"],
                "questions": [q for q in x["questions"] if q["template"] in id_templates],
                "ood_questions": [q for q in x["questions"] if q["template"] not in id_templates],
            }
            for x in id_country_ood_pair_test
        ], f"{save_dir_fullpath}/test_unseen_pairs_seen_country.jsonl")

        io.dump_jsonlines([
            {
                "text": x["text"],
                "questions": [q for q in x["questions"] if q["template"] in id_templates],
                "ood_questions": [q for q in x["questions"] if q["template"] not in id_templates],
            }
            for x in ood_country_test
        ], f"{save_dir_fullpath}/test_unseen_pairs_unseen_country.jsonl")




100%|██████████| 2/2 [01:03<00:00, 31.56s/it]



100%|██████████| 2/2 [02:14<00:00, 67.36s/it]


In [663]:
# syn_data[0]

In [248]:
# n_test = 100
# n_dev = 100
# n_train = n_data - n_test - n_dev

# rand_shuffle = np.arange(n_data)
# np.random.shuffle(rand_shuffle)

# train_data = [syn_data[i] for i in rand_shuffle[:n_train]]
# dev_data = [syn_data[i] for i in rand_shuffle[n_train:n_train+n_dev]]
# test_data = [syn_data[i] for i in rand_shuffle[n_train+n_dev:]]

# # io.dump_jsonlines(train_data, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/train.jsonl")
# # io.dump_jsonlines(dev_data, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/valid.jsonl")
# io.dump_jsonlines(test_data, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/test_ood.jsonl")

In [36]:
len(test_data)

100

In [3]:

# Define the regex pattern
pattern = r"(?P<first_name>\w+) (?P<last_name>\w+) was born in (?P<birth_place>[\w\s,'.()-]+)\. (?P<gender_start>\w+) started and completed a career of (?P<career>[\w\s]+) in (?P<career_place>[\w\s,'.()-]+)\. After retirement, (?P<gender>\w+) moved to (?P<death_place>[\w\s,'.()-]+) and passed away\."

# Example function to extract information
def extract_person_info_country(text):
    match = re.search(pattern, text)
    if match:
        return match.groupdict()
    return None

# Example usage
# text = "John Doe was born in New York City. He started and completed a career of engineering in California. After retirement, he moved to Florida and passed away."
# info = extract_person_info(text)
# print(info)

In [33]:
template_type2templates = dict([
    ("country", country_templates), 
    ("continent", continent_templates), 
    ("compare-city", compare_city_templates),
    ("compare-country", compare_country_templates),
    ("compare-continent", compare_continent_templates), 
])
test_data = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/test.jsonl")

In [ ]:
birth_ccc = (*birth_cc, country2continent[birth_cc[1]])
career_ccc = (*career_cc, country2continent[career_cc[1]])
death_ccc = (*death_cc, country2continent[death_cc[1]])

In [108]:
test_data[0]["text"]

'Charlotte Scott was born in Puerto Montt. She started and completed a career of Lawyer in Chelyabinsk. After retirement, she moved to Chelyabinsk and passed away.'

In [110]:
syn_info = extract_person_info_country(test_data[0]["text"])

In [130]:
new_test_data = []

for test_datum in test_data: 
    new_test_datum = deepcopy(test_datum)
    syn_info = extract_person_info_country(test_datum["text"])
    
    alternative_questions = []
    # try:
    birth_ccc = (syn_info["birth_place"], city2countries[syn_info["birth_place"]], country2continent[city2countries[syn_info["birth_place"]]])
    career_ccc = (syn_info["career_place"], city2countries[syn_info["career_place"]], country2continent[city2countries[syn_info["career_place"]]])
    death_ccc = (syn_info["death_place"], city2countries[syn_info["death_place"]], country2continent[city2countries[syn_info["death_place"]]])
    # except:
    #     break
    
    for q in test_datum["questions"]:
        templates = template_type2templates[q["question_type"]]
        instantiated_templates = [
            template[0].format(first_name=syn_info["first_name"], last_name=syn_info["last_name"], career=syn_info["career"],gender=syn_info["gender"])
            for template in templates
        ]
        
        assert len([e_i for e_i, t in enumerate(instantiated_templates) if q["question"] in t]) == 1
        assert len([e_i for e_i, t in enumerate(instantiated_templates) if q["question"] not in t]) == 2
        
        alternative_template_ids = [e_i for e_i, t in enumerate(instantiated_templates) if q["question"] not in t]
        alternative_template_idx = np.random.choice(alternative_template_ids)
        alternative_template = templates[alternative_template_idx]
        alternative_questions.append(
            {
                "question_type": q["question_type"],
                "question": alternative_template[0].format(
                    first_name=syn_info["first_name"],
                    last_name=syn_info["last_name"],
                    career=syn_info["career"],
                    gender=syn_info["gender"],
                ),
                "answer": str(alternative_template[1](birth_ccc[0], career_ccc[0], death_ccc[0]))
            }
        )
    new_test_datum["alternative_questions"] = deepcopy(alternative_questions)
    
    np.random.shuffle(alternative_questions)
    text_w_qas = test_datum["text"] + "\n\n" + "\n".join(f"{q['question']} {q['answer']}" for q in alternative_questions)
    new_test_datum["text_w_qas"] = text_w_qas
    
    new_test_data.append(new_test_datum)

In [134]:
print(new_test_data[0]["text_w_qas"])

Charlotte Scott was born in Puerto Montt. She started and completed a career of Lawyer in Chelyabinsk. After retirement, she moved to Chelyabinsk and passed away.

Did Charlotte Scott work and die in the same country? Yes
Was Charlotte Scott born and died in the same continent? No
Which continent did Charlotte Scott die in? Europe
Was Charlotte Scott born and worked in the same city? No
Which country did Charlotte Scott worked in? Russia


In [135]:
io.dump_jsonlines(new_test_data, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/test.jsonl")

In [24]:
ds = load_from_disk(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/border_countries.hf")

In [38]:
country2countries_pairs = {}

for i in range(len(ds)):
    country = ds[i]["country"]
    
    border_countries = list(set(ds[i]["bordering_countries"]))
    non_bordering_countries = list(set(ds[i]["non_bordering_countries"]))
    
    if len(border_countries) == 0 or len(non_bordering_countries) == 0:
        continue
    n_border_country = len(border_countries)
    n_non_border_country = len(non_bordering_countries)
    
    n_country = min(n_border_country, n_non_border_country)
    country2countries_pairs[country] = {
        "border": border_countries[:n_country], 
        "non_border": non_bordering_countries[:n_country]
    }
len(country2countries_pairs)

61

In [46]:
io.dump_json(country2countries_pairs, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/country2countries_pairs.json")

In [39]:

ood_country_events = [
    "{first_name} {last_name} was born",
    "{first_name} {last_name} worked in",
    "{first_name} {last_name} died in",
]

In [40]:
"Is {country} bordering with country where " + ood_country_events[0] + "?"

'Is {country} bordering with country where {first_name} {last_name} was born?'

In [ ]:
new_test_data = []

for test_datum in test_data[:]: 
    new_test_datum = deepcopy(test_datum)
    syn_info = extract_person_info_country(test_datum["text"])
    
    alternative_questions = []
    # try:
    birth_ccc = (syn_info["birth_place"], city2countries[syn_info["birth_place"]], country2continent[city2countries[syn_info["birth_place"]]])
    career_ccc = (syn_info["career_place"], city2countries[syn_info["career_place"]], country2continent[city2countries[syn_info["career_place"]]])
    death_ccc = (syn_info["death_place"], city2countries[syn_info["death_place"]], country2continent[city2countries[syn_info["death_place"]]])
    
    ood_qs = []
    
    for i, ccc in enumerate([birth_ccc, career_ccc, death_ccc]):
        
        if ccc[1] not in country2countries_pairs:
            continue
        
        border_countries = country2countries_pairs[ccc[1]]["border"]
        non_border_countries = country2countries_pairs[ccc[1]]["non_border"]
        ood_birth_template = "Is {country} bordering with country where " + ood_country_events[i] + "?"
        for b, nb in zip(border_countries, non_border_countries):
            
            b_q = ood_birth_template.format(first_name=syn_info["first_name"], last_name=syn_info["last_name"], country=b)
            nb_q = ood_birth_template.format(first_name=syn_info["first_name"], last_name=syn_info["last_name"], country=nb)
            ood_qs.append({"question": b_q, "answer": "Yes", "question_type": "border"})
            ood_qs.append({"question": nb_q, "answer": "No", "question_type": "non_border"})
    
    new_test_datum["ood_questions"] = deepcopy(ood_qs)
    new_test_data.append(new_test_datum)
        

In [61]:
sum([len(x["ood_questions"]) for x in new_test_data])

208

In [ ]:
# io.dump_jsonlines(new_test_data, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/test_ood.jsonl")

In [51]:
len(new_test_data)

100

# Genereate OOD questions before prompting GPT for answer (v2)

## for bio_syn_v2 dataset

In [15]:
year_ood_templates = [
    "Who was the U.S. President in {year}?",
    "Who was the Emperor of Japan in {year}?",
    "Who was the monarch of the United Kingdom in {year}?",
    "Who was the Prime Minister of the United Kingdom in {year}?",
    "Who won the Nobel Peace Prize in {year}?",
    "Who won the Nobel Prize in Physics in {year}?",
    "Who was the Pope in {year}?"
]
year_ood_templates_alias = [
    "Who was the U.S. President {year}?",
    "Who was the Emperor of Japan {year}?",
    "Who was the monarch of the United Kingdom {year}?",
    "Who was the Prime Minister of the United Kingdom {year}?",
    "Who won the Nobel Peace Prize {year}?",
    "Who won the Nobel Prize in Physics {year}?",
    "Who was the Pope {year}?"
]


In [10]:
import re

year_pattern = r"(?P<first_name>\w+) (?P<last_name>\w+) was born in (?P<birth_year>\d+) in (?P<birth_place>[\w\s]+)\. (?P<gender_start>\w+) started the career of (?P<career>[\w\s]+) in (?P<career_year>\d+)\. In (?P<death_year>\d+), (?P<gender>\w+) passed away\."

# Example usage
def extract_info_year(text):
    match = re.search(year_pattern, text)
    if match:
        return match.groupdict()
    return None

# Test with example text

In [4]:


# Define the regex pattern

place_pattern = r"(?P<first_name>\w+) (?P<last_name>\w+) was born in (?P<birth_place>[\w\s,'.()-]+)\. (?P<gender_start>\w+) started and completed a career of (?P<career>[\w\s]+) in (?P<career_place>[\w\s,'.()-]+)\. After retirement, (?P<gender>\w+) moved to (?P<death_place>[\w\s,'.()-]+) and passed away\."

# Example function to extract information
def extract_info_place(text):
    match = re.search(place_pattern, text)
    if match:
        return match.groupdict()
    return None

# Example usage
# text = "John Doe was born in 1950 in New York City. He started the career of engineering in 1972. In 2020, he passed away."
# info = extract_person_info(text)
# print(info)

In [8]:
bio_syn_data = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data_v2/test.jsonl")
country_syn_data = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/test.jsonl")


In [9]:
all_years = []
all_cities = []
for d in bio_syn_data:
    year_data_extraction = extract_info_year(d["text"])
    all_years.append(year_data_extraction["birth_year"])
    all_years.append(year_data_extraction["career_year"])
    all_years.append(year_data_extraction["death_year"])
    all_cities.append(year_data_extraction["birth_place"])

for d in country_syn_data:
    country_data_extraction = extract_info_place(d["text"])
    all_cities.append(country_data_extraction["birth_place"])
    all_cities.append(country_data_extraction["career_place"])
    all_cities.append(country_data_extraction["death_place"])
        

all_years = list(set(all_years))
all_cities = list(set(all_cities))

NameError: name 'extract_info_year' is not defined

In [6]:
# io.dump_jsonlines(all_cities, f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data_v2/test_birth_cities.jsonl")

In [24]:
test_countries = load_from_disk(f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data_v2/test_birth_country.hf")

In [14]:
test_city2country = dict(zip(test_countries["city"], test_countries["country"]))
test_city2country["Maldives"] = "Maldives"

In [26]:
city_country_pairs_train = io.load_json("/u/zliu/datastor1/KE-by-CP/data/debug_meta_train/country_syn_data/city_country_pairs_train.json")
city_country_pairs_valid = io.load_json("/u/zliu/datastor1/KE-by-CP/data/debug_meta_train/country_syn_data/city_country_pairs_valid.json")

In [29]:
city2country_pairs_train = {}
for city, country in city_country_pairs_train:
    city2country_pairs_train[city] = country
assert len(city2country_pairs_train) == len(city_country_pairs_train)
city2country_pairs_valid = {}
for city, country in city_country_pairs_valid:
    city2country_pairs_valid[city] = country
assert len(city2country_pairs_valid) == len(city_country_pairs_valid)

In [64]:
len(city2country_pairs_valid)

564

In [30]:
for k, v in city2country_pairs_train.items():
    if k in city2country_pairs_valid:
        assert v == city2country_pairs_valid[k]
    else:
        city2country_pairs_valid[k] = v

In [33]:
# io.dump_json(city2country_pairs_valid, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/city2country.json")

In [83]:
for c in all_cities:
    if c not in city2country:
        print(c)

In [ ]:
# city2country["Reykjavik"] = "Iceland"
# city2country["Marrakech"] = "Morocco"
# city2country["Queenstown"] = "New Zealand"
# city2country["Santorini"] = "Greek"
# city2country["Machu Picchu"] = "Peru"
# city2country["Havana"] = "Cuba"
# city2country["Maldives"] = "Maldives"
# city2country["New York City"] = "United States"
# city2country["Bora Bora"] = "French Polynesia"


In [91]:
# io.dump_json(city2country, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/city2country.json")

In [ ]:
# country2continent["French Polynesia"] = "Oceania"
# country2continent["Greek"] = "Europe"

In [ ]:
# io.dump_json(country2continent, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/country2continent.json")

In [90]:
# country2continent["French Polynesia"]

In [ ]:
# country2continent = io.load_json(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/country2continent_train.json")
# # manually adding some missing ones
# country2continent["Maldives"] = "Asia"
# country2continent["Iceland"] = "Europe"
# country2continent["Cuba"] = "North America"

# # io.dump_json(country2continent, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/country2continent.json")

In [20]:
# country2continent_valid = io.load_json(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/country2continent.json")

In [21]:
# for k, v in country2continent.items():
#     if k in country2continent_valid:
#         assert v == country2continent_valid[k]
#     else:
#         country2continent_valid[k] = v

In [5]:
# io.dump_json(country2continent_valid, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/country2continent.json")
city2country = io.load_json(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/city2country.json")
country2continent = io.load_json(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/country2continent.json")

In [25]:
# for k in test_city2country.values():
#     assert k in country2continent, k

In [6]:
place_ood_templates = [
    "What is the capital of {country}?",
    "What language in {country} has the most speakers?",
    "What is the currency of {country}?",
    "Which ethnic group is the largest in {country}?",
    "Which country is the largest in {continent} by area?",
    "Which religion has the most followers in {country}?"
]

place_ood_templates_v2 = [
    "What type of government (e.g., constitutional monarchy, federal republic) does {country} have?",
    "What is the national animal of {country}?",
    "What is the national flower of {country}?",
    "What is the national anthem of {country} called?",
    "What is the full official name of {country}?"
]

In [76]:
len(year_ood_templates)

NameError: name 'year_ood_templates' is not defined

In [11]:
all_year_question = []
all_place_question = []
key2questions = defaultdict(list)

# for yr in all_years:
#     for template in year_ood_templates:
#         question = template.format(year=yr)
#         key2questions[yr].append((question, template))
        
#         all_year_question.append(
#             {
#                 "question": question,
#             }
#         )

for city in all_cities:
    country = city2country[city]
    continent = country2continent[country]
    
    for template in place_ood_templates_v2:
        question = template.format(country=country, continent=continent)
        key2questions[country].append((question, template))
        all_place_question.append(
            {
                "question": question,
            }
        )

In [12]:
len(all_place_question)

0

In [110]:
key2questions = {k: list(set(v)) for k, v in key2questions.items()}

In [13]:
question_df_content = []
for k, v in key2questions.items():
    for q, t in v:
        question_df_content.append(
            {
                "question": q,
                "template": t,
                "key": k,
                "domain": "country" if k in country2continent else "year",
            }
        )
question_df = pd.DataFrame(question_df_content)

In [84]:
question_df.drop_duplicates(subset=["question"]).to_excel(f"{vars.DATA_DIR}/debug_meta_train/ood_country_question_df_v3.xlsx", index=False)

In [332]:
# io.dump_jsonlines(all_world_knowledge_question, f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data_v2/popular_ood_questions_v2.jsonl")
popular_ood_questions_ds = load_from_disk(f"{vars.DATA_DIR}/debug_meta_train/country_knowledge_questions_missing2.hf")

# # len(popular_ood_questions_ds)

In [333]:
len([x for x in popular_ood_questions_ds["answers"] if x == ["I don't know"]])

0

In [334]:
popular_ood_questions_df = popular_ood_questions_ds.to_pandas()

In [335]:
popular_ood_questions_ds.column_names

['question', 'template', 'key', 'domain', 'answers']

In [336]:
popular_ood_questions_ds[0]

{'question': 'What is the full official name of Bolivia?',
 'template': 'What is the full official name of {country}?',
 'key': 'Bolivia',
 'domain': 'country',
 'answers': ['Plurinational State of Bolivia']}

In [338]:
# all_questions = popular_ood_questions_ds["question"]
# Counter(all_questions)

In [337]:
# unique_questions = popular_ood_questions_ds.to_pandas().drop_duplicates(subset=["question",])["question"].tolist()

In [ ]:
question2answers = {}
bad_qa = defaultdict(list)
popular_ood_questions_df_content_processed = []


# for k, vs in zip(popular_ood_questions_ds["question"], popular_ood_questions_ds["answers"]):
for i in range(len(popular_ood_questions_ds)):
    datum = popular_ood_questions_ds[i]
    answers = datum["answers"]
    if datum["domain"] == "year":
        continue
    # if datum["template"] == "What is the full official name of {country}?":
        # continue
    if answers == ["I don't know"]:
        bad_qa["I don't know"].append(datum["question"])
        if datum["question"] == "What language has the most speakers in Greek?":
            datum["answers"] = ["Greek"]
            # print(datum)
        else:
            continue
        
    if datum["question"] in ["What is the national animal of Israel?", ]:
        continue
    elif len(answers) > 1:
        
        if datum['question'] in ["What is the national flower of United Kingdom?", "What is the full official name of Japan?", "What is the national flower of Indonesia?", "What is the capital of South Africa?", "What is the national flower of China?", "What is the national animal of Canada?", "What is the national anthem of Denmark called?", "What is the full official name of New Zealand?", "What is the national anthem of South Africa called?", "What is the full official name of Czech Republic?", "What is the full official name of Ireland?", "What is the currency of Cuba?", "What is the capital of Bolivia?", "What language has the most speakers in Uganda?"]:
            continue
        assert len(answers) == 1, datum["question"] + str(answers)
        
        bad_qa["Multiple Answer"].append(datum["question"])

    # manually fix factual errors
    if datum["question"] =="Which ethnic group is the largest in South Africa?":
        datum["answers"] = ["Black African"]
    elif datum["question"] == "Which ethnic group is the largest in South Korea?":
        datum["answers"] = ["Korean"]
    elif datum["question"] == "Which ethnic group is the largest in Morocco?":
        datum["answers"] = ["Arab"]
    elif datum["question"] == "Which ethnic group is the largest in Iceland?":
        datum["answers"] = ["Icelandic"]
    elif datum["question"] == "Which religion has the most followers in South Korea?":
        datum["answers"] = ["Christianity"]  
    elif datum["question"] == "Which religion has the most followers in Nigeria?":
        datum["answers"] = ["Christianity"]
    elif datum["question"] == "Which religion has the most followers in Japan?":
        datum["answers"] = ["Shinto"]
    elif datum["question"] == "What language has the most speakers in Malaysia?":
        datum["answers"] = ["Malay"]
    elif datum["question"] == "What language has the most speakers in Ghana?":
        datum["answers"] = ["Akan"]
    elif datum["question"] == "What is the national flower of China?":
        datum["answers"] = ["Plum Blossom"]
    elif datum["question"] == "What is the national animal of Canada?":
        datum["answers"] = ["Beaver"]
    elif datum["question"] == "What is the full official name of New Zealand?":
        datum["answers"] = ["New Zealand"]
    elif datum["question"] == "What is the national animal of Russia?":
        datum["answers"] = ["Eurasian brown bear"]
    elif datum["question"] == "What is the national animal of Germany?":
        datum["answers"] = ["The black eagle (Bundesadler)"]
    elif datum["question"] == "What is the national animal of Singapore?":
        datum["answers"] = ["Lion"]
    elif datum["question"] == "What is the national animal of Greece?":
        datum["answers"] = ["Dolphin"]
    elif datum["question"] == "What is the national animal of Armenia?":
        datum["answers"] = ["the Golden Eagle"]
    elif datum["question"] == "What is the full official name of Czech Republic?":
        datum["answers"] = ["the Czech Republic"]
    elif datum["question"] == "What is the full official name of Ireland?":
        datum["answers"] = ["the Republic of Ireland"]
    elif datum["question"] == "What is the full official name of Hungary?":
        datum["answers"] = ["the Republic of Hungary"]
    elif datum["question"] == "What is the full official name of Iceland?":
        datum["answers"] = ["the Republic of Iceland"]
    elif datum["question"] == "What is the full official name of Malaysia?":
        datum["answers"] = ["The Federation of Malaya"]
    elif datum["question"] == "What is the full official name of Canada?":
        datum["answers"] = ["Dominion of Canada"]
    elif datum["question"] == "What is the full official name of French Polynesia?":
        datum["answers"] = ["Overseas Country of French Polynesia"]
    elif datum["question"] == "What is the currency of Cuba?":
        datum["answers"] = ["Cuban Peso"]
    elif datum["question"] == "What is the full official name of Greek?":
        datum["answers"] = ["the Hellenic Republic"]
    elif datum["question"] == "Which ethnic group is the largest in Cuba?":
        datum["answers"] = ["White Cubans"]
    elif datum["question"] == "Which ethnic group is the largest in French Polynesia?":
        datum["answers"] = ["Tahitians (Polynesians)"]
    elif datum["question"] == "Which religion has the most followers in Iceland?":
        datum["answers"] = ["Church of Iceland (Lutheran)"]
        
    elif datum["question"] == "Which religion has the most followers in Greek?":
        datum["answers"] = ["Eastern Orthodox Church"]
    elif datum["question"] == "What language has the most speakers in Uganda?":
        datum["answers"] = ["English"]
        
    
    popular_ood_questions_df_content_processed.append(
        {
            "question": datum["question"],
            "answer": ", ".join(datum["answers"]),
            "template": datum["template"],
            "key": datum["key"],
            "domain": datum["domain"],
        }
    )
    # question2answers[k] = vs
# io.dump_json(question2answers, f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data_v2/popular_ood_question2answers.json")
# question2answers = io.load_json(f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data_v2/popular_ood_question2answers.json")
print("Count of 'I don't know' answers:", len(bad_qa["I don't know"]))
print("Count of multiple answers:", len(bad_qa["Multiple Answer"]))

popular_ood_questions_df = pd.DataFrame(popular_ood_questions_df_content_processed)

len(popular_ood_questions_df_content_processed) == len(popular_ood_questions_df.drop_duplicates(subset=["question", ]))


Count of 'I don't know' answers: 0
Count of multiple answers: 0


False

In [342]:
# country_v3_df = pd.read_excel("/datastor1/zliu/mend/debug_exp_output/llama3.2-1B-eos-sft/ood_v3_prefilter/base_n=291_prompt=no_w-gen_wo-icl_ice=False.xlsx")
# for r_i, r in country_v3_df[country_v3_df["template"] == "What is the full official name of {country}?"].iterrows():
#     if r["llm_accuracy"] < 1.0:
#         print(r["question"])
#         print(r["answer"])
#         print()

In [344]:

for i, r in popular_ood_questions_df[popular_ood_questions_df["template"] == country_knowledge_template_v3[0]].iterrows():
    print(r["question"])
    print(r["answer"])
    print()

What is the full official name of Bolivia?
Plurinational State of Bolivia

What is the full official name of Democratic Republic of the Congo?
Democratic Republic of the Congo

What is the full official name of Ethiopia?
Federal Democratic Republic of Ethiopia

What is the full official name of Georgia?
Georgia

What is the full official name of Mozambique?
Republic of Mozambique

What is the full official name of Oman?
Sultanate of Oman

What is the full official name of Qatar?
State of Qatar

What is the full official name of Uganda?
Republic of Uganda

What is the full official name of Uruguay?
Oriental Republic of Uruguay

What is the full official name of Zambia?
Republic of Zambia

What is the full official name of Zimbabwe?
Republic of Zimbabwe



In [281]:
len(popular_ood_questions_df)

34

In [282]:
len(popular_ood_questions_df.drop_duplicates(subset=['question'], keep='first'))

33

In [159]:
# popular_ood_questions_df[popular_ood_questions_df["template"] == "What is the full official name of {country}?"]

In [ ]:
# io.dump_jsonlines(popular_ood_questions_df_content_processed, f"{vars.DATA_DIR}/debug_meta_train/ood_country_questions_v3_missing2.jsonl")

In [180]:
for template in (year_ood_templates + place_ood_templates):
    print(template)
    unique_answeres = set(popular_ood_questions_df[popular_ood_questions_df["template"] == template]["answer"].unique())
    print("# unique answer: ", len(unique_answeres))
    print(unique_answeres)
    print()

Who was the U.S. President in {year}?
# unique answer:  28
{'Richard Nixon', 'George H. W. Bush', 'Donald Trump', 'Richard Nixon, Gerald Ford', 'Herbert Hoover', 'John F. Kennedy, Lyndon B. Johnson', 'George W. Bush', 'Franklin D. Roosevelt', 'Franklin D. Roosevelt, Harry S. Truman', 'Lyndon B. Johnson', 'Warren G. Harding, Calvin Coolidge', 'Barack Obama', 'John F. Kennedy', 'Theodore Roosevelt', 'Dwight D. Eisenhower, John F. Kennedy', 'Bill Clinton', 'Jimmy Carter', 'Dwight D. Eisenhower', 'Gerald Ford', 'William Howard Taft', 'William McKinley', 'Harry S. Truman', 'Calvin Coolidge', 'Warren G. Harding', 'Woodrow Wilson', 'Joe Biden', 'William McKinley, Theodore Roosevelt', 'Ronald Reagan'}

Who was the Emperor of Japan in {year}?
# unique answer:  10
{'Emperor Shōwa', 'Emperor Meiji', 'Akihito', 'Emperor Shōwa (Hirohito)', 'Emperor Showa (Hirohito)', 'Emperor Taishō', 'Emperor Akihito', 'Emperor Hirohito', 'Naruhito', 'Hirohito'}

Who was the monarch of the United Kingdom in {year}

In [114]:
# io.dump_jsonlines(popular_ood_questions_df.to_dict('records'), f"{vars.DATA_DIR}/debug_meta_train/ood_country_questions_v3.jsonl")

In [161]:
country_questions = io.load_jsonlines("/u/zliu/datastor1/KE-by-CP/data/debug_meta_train/common_country_data/train_continent.jsonl") + io.load_jsonlines("/u/zliu/datastor1/KE-by-CP/data/debug_meta_train/common_country_data/valid_continent.jsonl")

In [167]:

# Example function to extract information
def extract_city(text):
    match = re.search(r"What country is (?P<city>[\w\s\-]+) in\?", text)
    if match:
        return match.groupdict()
    return None


# Example function to extract information
def extract_country(text):
    match = re.search(r"Which continent is (?P<country>[\w\s\-\'\.]+) located in\?", text)
    if match:
        return match.groupdict()
    return None

In [168]:
country_questions[0]

{'question': 'Which continent is Ghana located in?', 'answer': 'Africa'}

In [178]:
# filtered_country_questions = []

# for q in country_questions:
#     if any(x in q["question"] for x in ["What country is The CBD (Central Business District) in?"]):
#         continue
#     assert extract_city(q["question"]) is not None, q["question"]
#     new_q = deepcopy(q)
#     city, country = q["(city, country)"] 
#     new_q["template"] = "Which country is {city} located in?"
#     new_q["domain"] = "country"
#     new_q["question"] = f"Which country is {city} located in?"
    
#     new_q["key"] = country
#     filtered_country_questions.append(new_q)

filtered_continent_questions = []

for q in country_questions:
    # if any(x in q["question"] for x in ["What country is The CBD (Central Business District) in?"]):
    #     continue
    assert extract_country(q["question"]) is not None, q["question"]
    new_q = deepcopy(q)
    country =  extract_country(q["question"])["country"]
    # city, country = q["(city, country)"] 
    new_q["template"] = "Which continent is {country} located in?"
    new_q["domain"] = "country"
    # new_q["question"] = f"Which country is {city} located in?"
    
    new_q["key"] = country
    filtered_continent_questions.append(new_q)
    

## some additional processing for missing countries

mostly just repeat some code above

In [176]:
len(filtered_continent_questions)

100

In [179]:
len(pd.DataFrame(filtered_continent_questions)["key"].unique())
# TODO: make sure all countries has similar # of questions

72

In [180]:
filtered_continent_questions

[{'question': 'Which continent is Ghana located in?',
  'answer': 'Africa',
  'template': 'Which continent is {country} located in?',
  'domain': 'country',
  'key': 'Ghana'},
 {'question': 'Which continent is Malaysia located in?',
  'answer': 'Asia',
  'template': 'Which continent is {country} located in?',
  'domain': 'country',
  'key': 'Malaysia'},
 {'question': 'Which continent is Philippines located in?',
  'answer': 'Asia',
  'template': 'Which continent is {country} located in?',
  'domain': 'country',
  'key': 'Philippines'},
 {'question': 'Which continent is Argentina located in?',
  'answer': 'South America',
  'template': 'Which continent is {country} located in?',
  'domain': 'country',
  'key': 'Argentina'},
 {'question': 'Which continent is New Zealand located in?',
  'answer': 'Oceania',
  'template': 'Which continent is {country} located in?',
  'domain': 'country',
  'key': 'New Zealand'},
 {'question': 'Which continent is China located in?',
  'answer': 'Asia',
  't

In [ ]:
# io.dump_jsonlines(filtered_continent_questions, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/country2continent_questions.jsonl")

unique_countries = pd.DataFrame(io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/city2country_questions.jsonl"))["key"].unique().tolist()

In [188]:
country_knowledge_countries = pd.DataFrame(io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/country_knowledge_questions.jsonl"))["key"].unique().tolist()

In [195]:
pd.DataFrame(io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/country_knowledge_questions.jsonl"))["template"].unique()

array(['What is the full official name of {country}?',
       'What is the capital of {country}?',
       'What language has the most speakers in {country}?',
       'What is the currency of {country}?',
       'Which ethnic group is the largest in {country}?',
       'Which country is the largest in {continent} by area?',
       'Which religion has the most followers in {country}?'],
      dtype=object)

In [190]:
missing_countries = [x for x in country_knowledge_countries if x not in unique_countries]

In [191]:
missing_countries

['Iceland', 'Cuba', 'Greek', 'Maldives', 'French Polynesia']

In [202]:
country_knowledge_template_v3 = [
    'What is the full official name of {country}?',
    'What is the capital of {country}?',
    'What language has the most speakers in {country}?',
    'What is the currency of {country}?',
    'Which ethnic group is the largest in {country}?',
    'Which country is the largest in {continent} by area?',
    'Which religion has the most followers in {country}?'
]

In [328]:

question_df_content = []
for c in ['Bolivia',
 'Democratic Republic of the Congo',
 'Ethiopia',
 'Georgia',
 'Mozambique',
 'Oman',
 'Qatar',
 'Uganda',
 'Uruguay',
 'Zambia',
 'Zimbabwe']:
    continent = country2continent[c]
    
    for t in country_knowledge_template_v3:
        q = t.format(country=c, continent=continent)
        question_df_content.append(
            {
                "question": q,
                "template": t,
                "key": c,
                "domain": "country",
            }
        )
question_df = pd.DataFrame(question_df_content)

In [330]:
question_df_content[0]

{'question': 'What is the full official name of Bolivia?',
 'template': 'What is the full official name of {country}?',
 'key': 'Bolivia',
 'domain': 'country'}

In [331]:
io.dump_jsonlines(question_df_content, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/country_knowledge_questions_missing2.jsonl")

# popular_ood_questions_ds = load_from_disk(f"{vars.DATA_DIR}/debug_meta_train/country_knowledge_questions_missing.hf")
# popular_ood_questions_df = popular_ood_questions_ds.to_pandas()

In [506]:
# io.dump_jsonlines(io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/subparts/city2country_questions.jsonl") + io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/subparts/country2continent_questions.jsonl") + io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/subparts/country_knowledge_questions.jsonl") + io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/subparts/missing_entries.jsonl"), f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/all_country_questions.jsonl")

In [507]:
country_knowledge_data_v2 = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/all_country_questions.jsonl")

# some weirdly missing entries

len(country_knowledge_data_v2)

1189

In [ ]:
# io.dump_jsonlines(country_knowledge_data_v2_missing, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/subparts/missing_entries.jsonl")

In [505]:
# remove duplicates after adding missing ones
# io.dump_jsonlines(pd.DataFrame(country_knowledge_data_v2).drop_duplicates(subset=["key", "question", ]).to_dict("records"), f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/all_country_questions.jsonl")

In [ ]:
# io.dump_jsonlines(country_knowledge_data_v2, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data_v2/all_country_questions.jsonl")

In [508]:
country2template_count = defaultdict(lambda : defaultdict(int))
for (country, template), df in pd.DataFrame(pd.DataFrame(country_knowledge_data_v2).drop_duplicates(subset=["key", "question", "answer", "template",  "domain", ], keep="first", inplace=False)).groupby(["key", "template"]):
    # print((country, template))
    # print(df.shape)
    country2template_count[country][template] += df.shape[0]

In [509]:
# len(pd.DataFrame(country_knowledge_data_v2).drop_duplicates(subset=["key", "question", "answer", "template",  "domain", ], keep="first", inplace=False))

In [510]:
country2template_count

defaultdict(<function __main__.<lambda>()>,
            {'Algeria': defaultdict(int,
                         {'What is the capital of {country}?': 1,
                          'What is the currency of {country}?': 1,
                          'What is the full official name of {country}?': 1,
                          'What language has the most speakers in {country}?': 1,
                          'Which continent is {country} located in?': 1,
                          'Which country is the largest in {continent} by area?': 1,
                          'Which country is {city} located in?': 1,
                          'Which ethnic group is the largest in {country}?': 1,
                          'Which religion has the most followers in {country}?': 1}),
             'Argentina': defaultdict(int,
                         {'What is the capital of {country}?': 1,
                          'What is the currency of {country}?': 1,
                          'What is the full official na

In [473]:
# pd.DataFrame(country_knowledge_data_v2,).dropna(subset=["answer"])

In [474]:
# io.dump_jsonlines(df.dropna().to_dict("records"), "/u/zliu/datastor1/KE-by-CP/data/debug_meta_train/country_syn_data_v2/country_knowledge_questions.jsonl")

In [475]:
# country2template_count

In [511]:

# for i, r in popular_ood_questions_df[popular_ood_questions_df["template"] == place_ood_templates_v2[-1]].iterrows():
#     print(r["question"])
#     print(r["answer"])
#     print()
# all([all(v == 1 for v in vs.values()) for k, vs in country2template_count.items()]) # each template has 1 instance for each country

[k  for k, vs in country2template_count.items() if len(vs) < 9] # each template has 1 instance for each country

['Bolivia', 'South Africa']

[{'question': 'What is the full official name of Japan?',
  'answer': 'Nippon-koku',
  'template': 'What is the full official name of {country}?',
  'key': 'Japan',
  'domain': 'country'},
 {'question': 'What is the full official name of New Zealand?',
  'answer': 'New Zealand (Aotearoa)',
  'template': 'What is the full official name of {country}?',
  'key': 'New Zealand',
  'domain': 'country'},
 {'question': 'What is the full official name of Ireland?',
  'answer': 'Republic of Ireland',
  'template': 'What is the full official name of {country}?',
  'key': 'Ireland',
  'domain': 'country'},
 {'question': 'What language has the most speakers in Uganda?',
  'answer': 'English',
  'template': 'What language has the most speakers in {country}?',
  'key': 'Uganda',
  'domain': 'country'},
 {'question': 'Which continent is Cuba located in?',
  'answer': 'North America',
  'template': 'Which continent is {country} located in?',
  'key': 'Cuba',
  'domain': 'country'},
 {'question': 'Which

In [ ]:
# df = pd.DataFrame(io.load_jsonlines("/u/zliu/datastor1/KE-by-CP/data/debug_meta_train/country_syn_data_v2/country_knowledge_questions.jsonl"))
len(df)

566

In [371]:
# len(df.drop_duplicates(subset=["key", "template",], keep="first"))

# add OOD_V2 questions to year_data

In [39]:
df = pd.read_excel("/datastor1/zliu/mend/debug_exp_output/llama3.2-1B-common-date-year-after-eos-sft/ood_v2_prefilter/base_n=1085_prompt=no_w-gen_wo-icl_ice=False.xlsx")
known_knowledge = set(df[df["llm_accuracy"] > 0.5]["question"].to_list())
question2answers = {}
for i, r in popular_ood_questions_df.iterrows():
    if r["question"] in known_knowledge:
        question2answers[r["question"]] = r["answer"]

In [41]:
# known_knowledge

In [274]:
# df[df["llm_accuracy"] > 0.5].describe()

In [42]:
events = [
    "{first_name} {last_name} was born",
    "{first_name} {last_name} started the career of {career}", 
    "{first_name} {last_name} passed away",
]

ood_arithmetic_offsets = [4, 5, 11, 13, 14, 15, 16]

year_ood_arithmetic_templates = [
    
]

def h(i, rel, memo=None):
    assert type(memo) == int
    if rel == "before":
        return lambda *x: x[i] - memo
    else:
        assert rel == "after"
        return lambda *x: x[i] + memo
        
for offset in ood_arithmetic_offsets:
    o = deepcopy(offset)
    
    year_ood_arithmetic_templates.extend(
        [
            (f"plus_{o}", f"When was {o} years after the year that " + events[0] + "?", h(0, rel="after", memo=deepcopy(offset))),
            (f"plus_{o}", f"When was {o} years after the year that " + events[1] + "?", h(1, rel="after", memo=deepcopy(offset))),
            (f"plus_{o}", f"When was {o} years after the year that " + events[2] + "?", h(2, rel="after", memo=deepcopy(offset))),
        ] + [
            (f"minus_{o}", f"When was {o} years before the year that " + events[0] + "?", h(0, rel="before", memo=deepcopy(offset))),
            (f"minus_{o}", f"When was {o} years before the year that " + events[1] + "?", h(1, rel="before", memo=deepcopy(offset))),
            (f"minus_{o}", f"When was {o} years before the year that " + events[2] + "?", h(2, rel="before", memo=deepcopy(offset))),
        ]
    )
    

In [58]:


bio_syn_data = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data_v2/test.jsonl")
bio_syn_data_w_ood = []

unanswerable_questions = set()
year_questions = []
place_questions = []
# dup_questions = set()

for t_d in bio_syn_data[:]:
    year_data_extraction = extract_info_year(t_d["text"])
    ood_questions = []
    for t in year_ood_arithmetic_templates:
        question = t[1].format(first_name=year_data_extraction["first_name"], last_name=year_data_extraction["last_name"], career=year_data_extraction["career"])
        answer = t[-1](
            int(year_data_extraction["birth_year"]),
            int(year_data_extraction["career_year"]),
            int(year_data_extraction["death_year"])
        )
        if "plus" in t[0]:
            rel = "after"
        else:
            assert "minus" in t[0]
            rel = "before"
        
        
        if "born" in question:
            yr = year_data_extraction["birth_year"]
        elif "career" in question:
            yr = year_data_extraction["career_year"]
        else:
            assert "passed away" in question, question
            yr = year_data_extraction["death_year"]
        
        offset = answer - int(yr) if rel == "after" else int(yr) - answer
        unaliased_question = "When is " + str(offset) + " years " + rel + " the year " + yr + "?"
        ood_questions.append({"question": question, "answer": str(answer), "question_type": t[0], "unaliased_question": unaliased_question})
    year_tmp = []
    for y_i, yr_type in enumerate(["birth_year", "career_year", "death_year"]):
        for t_i, t in enumerate(year_ood_templates):
            
            question = t.format(year=year_data_extraction[yr_type])
            if question not in question2answers:
                unanswerable_questions.add(question)
                continue
            answers = question2answers[question]
            alias_yr = "when " + events[y_i].format(first_name=year_data_extraction["first_name"], last_name=year_data_extraction["last_name"], career=year_data_extraction["career"])
            
            aliased_question = year_ood_templates_alias[t_i].format(year=alias_yr)
            ood_questions.append({"question": aliased_question, "answer": answers, "question_type": f"common_fact_{yr_type}", "unaliased_question": question})
            year_tmp.append(1)
    year_questions.append(year_tmp)
    
    birth_country = city2country[year_data_extraction["birth_place"]]
    continent = country2continent[birth_country]
    place_tmp = []
    for t in place_ood_templates:
        
        question = t.format(country=birth_country, continent=continent)
        if question not in question2answers:
            
            continue
        answers = question2answers[question]
        aliased_country = "the country where " + events[0].format(first_name=year_data_extraction["first_name"], last_name=year_data_extraction["last_name"])
        
        aliased_continent = "the continent where " + events[0].format(first_name=year_data_extraction["first_name"], last_name=year_data_extraction["last_name"])
        aliased_question = t.format(country=aliased_country, continent=aliased_continent)
        ood_questions.append({"question": aliased_question, "answer": answers, "question_type": f"common_fact_birth_place", "unaliased_question": question})
        place_tmp.append(1)
    place_questions.append(place_tmp)
    
    t_d["ood_questions"] = ood_questions
    assert all("unaliased_question" in q for q in ood_questions)
    bio_syn_data_w_ood.append(t_d)
    
    

In [48]:
describe([len(x["ood_questions"]) for x in bio_syn_data_w_ood])

DescribeResult(nobs=100, minmax=(np.int64(51), np.int64(61)), mean=np.float64(56.84), variance=np.float64(3.933737373737373), skewness=np.float64(-0.3469730446776399), kurtosis=np.float64(0.24761999447704097))

In [60]:
describe([len(x) for x in place_questions])

DescribeResult(nobs=100, minmax=(np.int64(1), np.int64(4)), mean=np.float64(2.99), variance=np.float64(0.49484848484848487), skewness=np.float64(-0.33626036599700865), kurtosis=np.float64(0.02119957245358073))

In [62]:
sum([len(x) for x in place_questions])

299

In [61]:
describe([len(x) for x in year_questions])

DescribeResult(nobs=100, minmax=(np.int64(7), np.int64(16)), mean=np.float64(11.85), variance=np.float64(3.3207070707070705), skewness=np.float64(-0.22937669443659287), kurtosis=np.float64(-0.11266434385346047))

In [63]:
sum([len(x) for x in year_questions])

1185

In [307]:
# bio_syn_data_w_ood[0]["questions"]
len(bio_syn_data_w_ood)

100

In [ ]:
# io.dump_jsonlines(bio_syn_data_w_ood, f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data_v2/test_ood.jsonl")

In [55]:
bio_syn_data_w_ood[0]["ood_questions"]

[{'question': 'When was 4 years after the year that Laura Roberts was born?',
  'answer': '1941',
  'question_type': 'plus_4',
  'unaliased_question': 'When is 4 years after the year 1937?'},
 {'question': 'When was 4 years after the year that Laura Roberts started the career of Physical Therapist?',
  'answer': '1959',
  'question_type': 'plus_4',
  'unaliased_question': 'When is 4 years after the year 1955?'},
 {'question': 'When was 4 years after the year that Laura Roberts passed away?',
  'answer': '1997',
  'question_type': 'plus_4',
  'unaliased_question': 'When is 4 years after the year 1993?'},
 {'question': 'When was 4 years before the year that Laura Roberts was born?',
  'answer': '1933',
  'question_type': 'minus_4',
  'unaliased_question': 'When is 4 years before the year 1937?'},
 {'question': 'When was 4 years before the year that Laura Roberts started the career of Physical Therapist?',
  'answer': '1951',
  'question_type': 'minus_4',
  'unaliased_question': 'When is 

In [251]:
year_ood_arithmetic_templates[3]

('minus_4',
 'When was 4 years before the year that {first_name} {last_name} was born?',
 <function __main__.h.<locals>.<lambda>(*x)>)

In [252]:
year_ood_arithmetic_templates[3][-1](2023, 2024, 2025)

2019

In [303]:


# bio_syn_data = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data_v2/test.jsonl")
# bio_syn_data_w_ood = []

# unanswerable_questions = set()

# for t_d in bio_syn_data[:]:
#     year_data_extraction = extract_info_year(t_d["text"])
#     ood_questions = []
#     for t in year_ood_arithmetic_templates:
#         question = t[1].format(first_name=year_data_extraction["first_name"], last_name=year_data_extraction["last_name"], career=year_data_extraction["career"])
#         answer = t[-1](
#             int(year_data_extraction["birth_year"]),
#             int(year_data_extraction["career_year"]),
#             int(year_data_extraction["death_year"])
#         )
#         if "plus" in t[0]:
#             rel = "after"
#         else:
#             assert "minus" in t[0]
#             rel = "before"
        
        
#         if "born" in question:
#             yr = year_data_extraction["birth_year"]
#         elif "career" in question:
#             yr = year_data_extraction["career_year"]
#         else:
#             assert "passed away" in question, question
#             yr = year_data_extraction["death_year"]
        
#         offset = answer - int(yr) if rel == "after" else int(yr) - answer
#         unaliased_question = "When is " + str(offset) + " years " + rel + " the year " + yr + "?"
#         ood_questions.append({"question": question, "answer": str(answer), "question_type": t[0], "unaliased_question": unaliased_question})
    
#     for y_i, yr_type in enumerate(["birth_year", "career_year", "death_year"]):
#         for t in year_ood_templates:
            
#             question = t.format(year=year_data_extraction[yr_type])
#             if question not in question2answers:
#                 unanswerable_questions.add(question)
#                 continue
#             answers = question2answers[question]
#             alias_yr = "when " + events[y_i].format(first_name=year_data_extraction["first_name"], last_name=year_data_extraction["last_name"], career=year_data_extraction["career"])
            
#             aliased_question = t.format(year=alias_yr)
#             ood_questions.append({"question": aliased_question, "answer": answers, "question_type": f"common_fact_{yr_type}", "unaliased_question": question})
    
#     birth_country = test_city2country[year_data_extraction["birth_place"]]
#     continent = country2continent[birth_country]
    
#     for t in place_ood_templates:
        
#         question = t.format(country=birth_country, continent=continent)
#         if question not in question2answers:
            
#             continue
#         answers = question2answers[question]
#         aliased_country = "the country where " + events[0].format(first_name=year_data_extraction["first_name"], last_name=year_data_extraction["last_name"])
        
#         aliased_continent = "the continent where " + events[0].format(first_name=year_data_extraction["first_name"], last_name=year_data_extraction["last_name"])
#         aliased_question = t.format(country=aliased_country, continent=aliased_continent)
#         ood_questions.append({"question": aliased_question, "answer": answers, "question_type": f"common_fact_birth_place", "unaliased_question": question})
    
#     t_d["ood_questions"] = ood_questions
#     assert all("unaliased_question" in q for q in ood_questions)
#     bio_syn_data_w_ood.append(t_d)
    
    

In [255]:
bio_syn_data_w_ood[0]["ood_questions"]

[{'question': 'When was 4 years after the year that Laura Roberts was born?',
  'answer': '1941',
  'question_type': 'plus_4',
  'unaliased_question': 'When is 4 years after the year 1937?'},
 {'question': 'When was 4 years after the year that Laura Roberts started the career of Physical Therapist?',
  'answer': '1959',
  'question_type': 'plus_4',
  'unaliased_question': 'When is 4 years after the year 1955?'},
 {'question': 'When was 4 years after the year that Laura Roberts passed away?',
  'answer': '1997',
  'question_type': 'plus_4',
  'unaliased_question': 'When is 4 years after the year 1993?'},
 {'question': 'When was 4 years before the year that Laura Roberts was born?',
  'answer': '1933',
  'question_type': 'minus_4',
  'unaliased_question': 'When is 4 years before the year 1937?'},
 {'question': 'When was 4 years before the year that Laura Roberts started the career of Physical Therapist?',
  'answer': '1951',
  'question_type': 'minus_4',
  'unaliased_question': 'When is 

In [77]:
len(all_years)

103

In [78]:
641 / 103

6.223300970873787

In [52]:
"2003" in all_years

True

In [79]:
len(unanswerable_questions)

0

In [84]:
# question2answers

In [85]:
len(bio_syn_data_w_ood)

100

In [ ]:
# io.dump_jsonlines(bio_syn_data_w_ood, f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data_v2/test_ood.jsonl")

In [216]:
# bio_syn_data_w_ood[0]

## for Country_syn OOD v2

In [356]:
df = pd.read_excel("/datastor1/zliu/mend/debug_exp_output/llama3.2-1B-common-country-eos-sft/ood_v2_prefilter/base_n=1085_prompt=no_w-gen_wo-icl_ice=False.xlsx")
known_knowledge = set(df[df["llm_accuracy"] > 0.5]["question"].to_list())
question2answers = {}
for i, r in popular_ood_questions_df.iterrows():
    if r["question"] in known_knowledge:
        question2answers[r["question"]] = r["answer"]

In [291]:
# question2answers

In [360]:

country_syn_data = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/test.jsonl")
# country_syn_data = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/test_w_ood_country.jsonl")

city2country = dict(io.load_json(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/city2country.json"))
country2continent = io.load_json(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/country2continent.json")

In [301]:
# Counter(city2country.values())

In [361]:
country_syn_data_w_ood = []
count = 0
for t_d in country_syn_data[:]:
    place_data_extraction = extract_info_place(t_d["text"])
    assert place_data_extraction["birth_place"] in city2country
    assert place_data_extraction["career_place"] in city2country
    assert place_data_extraction["death_place"] in city2country
    birth_country = city2country[place_data_extraction["birth_place"]]
    career_country = city2country[place_data_extraction["career_place"]]
    death_country = city2country[place_data_extraction["death_place"]]
    
    assert birth_country in country2continent
    assert career_country in country2continent
    assert death_country in country2continent
    
    birth_continent = country2continent[birth_country]
    career_continent = country2continent[career_country]
    death_continent = country2continent[death_country]
    ood_questions = []
    dup_questions = set()
    
    for p_i, (p_name, ctry, cont) in enumerate([("birth_place", birth_country, birth_continent), ("career_place", career_country, career_continent), ("death_place", death_country, death_continent)]):
        for t in place_ood_templates:
        
            question = t.format(country=ctry, continent=cont)
            if question not in question2answers:
                count += 1
                continue
            answers = question2answers[question]
            aliased_country = "the country where " + events[p_i].format(first_name=place_data_extraction["first_name"], last_name=place_data_extraction["last_name"], career=place_data_extraction["career"])
            
            aliased_continent = "the continent where " + events[p_i].format(first_name=place_data_extraction["first_name"], last_name=place_data_extraction["last_name"], career=place_data_extraction["career"])
            aliased_question = t.format(country=aliased_country, continent=aliased_continent)
            if aliased_question in dup_questions:
                continue
            dup_questions.add(aliased_question)
            ood_questions.append({"question": aliased_question, "answer": answers, "question_type": f"common_fact_{p_name}", "unaliased_question": question})
    
    t_d["ood_questions"] = ood_questions
    country_syn_data_w_ood.append(t_d)
    
# io.dump_jsonlines(country_syn_data_w_ood, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/test_ood_w_ood_country.jsonl")
# io.dump_jsonlines(country_syn_data_w_ood, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/test_ood.jsonl")

In [362]:
describe([len(x["ood_questions"]) for x in country_syn_data_w_ood])

DescribeResult(nobs=100, minmax=(np.int64(8), np.int64(15)), mean=np.float64(11.23), variance=np.float64(2.1990909090909097), skewness=np.float64(-0.11965404846728482), kurtosis=np.float64(-0.2973747929382853))

In [371]:
io.load_jsonlines("/u/zliu/datastor1/KE-by-CP/data/debug_meta_train/bio_syn_data_v2/test_ood.jsonl")[0]["ood_questions"]

[{'question': 'When was 4 years after the year that Laura Roberts was born?',
  'answer': '1941',
  'question_type': 'plus_4',
  'unaliased_question': 'When is 4 years after the year 1937?'},
 {'question': 'When was 4 years after the year that Laura Roberts started the career of Physical Therapist?',
  'answer': '1959',
  'question_type': 'plus_4',
  'unaliased_question': 'When is 4 years after the year 1955?'},
 {'question': 'When was 4 years after the year that Laura Roberts passed away?',
  'answer': '1997',
  'question_type': 'plus_4',
  'unaliased_question': 'When is 4 years after the year 1993?'},
 {'question': 'When was 4 years before the year that Laura Roberts was born?',
  'answer': '1933',
  'question_type': 'minus_4',
  'unaliased_question': 'When is 4 years before the year 1937?'},
 {'question': 'When was 4 years before the year that Laura Roberts started the career of Physical Therapist?',
  'answer': '1951',
  'question_type': 'minus_4',
  'unaliased_question': 'When is 

In [ ]:
    
country_syn_all_question = []
country_key2questions = defaultdict(list)
dup_questions = set()

for t_d in country_syn_data[:]:
    place_data_extraction = extract_person_info_country(t_d["text"])
    assert place_data_extraction["birth_place"] in city2country
    assert place_data_extraction["career_place"] in city2country
    assert place_data_extraction["death_place"] in city2country
    birth_country = city2country[place_data_extraction["birth_place"]]
    career_country = city2country[place_data_extraction["career_place"]]
    death_country = city2country[place_data_extraction["death_place"]]
    
    assert birth_country in country2continent
    assert career_country in country2continent
    assert death_country in country2continent
    
    birth_continent = country2continent[birth_country]
    career_continent = country2continent[career_country]
    death_continent = country2continent[death_country]
    
    for ctry, cont in [ (birth_country, birth_continent), (career_country, career_continent), (death_country, death_continent)]:

        for template in place_ood_templates:
            question = template.format(country=ctry, continent=cont)
            country_key2questions[ctry].append(template)
            if question in dup_questions:
                continue
            
            dup_questions.add(question)
            country_syn_all_question.append(
                {
                    "question": question,
                }
            )
    

In [298]:
country_syn_all_question

NameError: name 'country_syn_all_question' is not defined

In [67]:
# len(country_syn_all_question)

In [293]:
len(set([x["question"] for x in country_syn_all_question]))

378

In [ ]:

# io.dump_jsonlines(country_syn_all_question, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/popular_ood_questions_valid.jsonl")

In [56]:
popular_ood_questions_ds = load_from_disk(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/popular_ood_questions_answers_valid.hf")

In [57]:
len([x["question"] for x in popular_ood_questions_ds if x["answers"] == ["I don't know"]])

4

In [79]:
question2answers = {}

# for k, vs in zip(popular_ood_questions_ds["question"], popular_ood_questions_ds["answers"]):
#     if vs == ["I don't know"]:
#         continue
#     question2answers[k] = vs
# io.dump_json(question2answers, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/popular_ood_question2answers_valid.json")
question2answers = io.load_json(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/popular_ood_question2answers_train.json")

In [64]:
# country_syn_data_w_ood[0]

In [ ]:
# country_syn_data_w_ood[0]["text"]

'Elizabeth Harris was born in Nakuru. She started and completed a career of Veterinarian in East London. After retirement, she moved to Nakuru and passed away.'

In [32]:
common_question_train_qa = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/common_country_data/valid.jsonl")
country2continent = io.load_json(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/country2continent_valid.json")
all([q["(city, country)"][-1] in country2continent for q in common_question_train_qa])

True

In [33]:
Counter(country2continent.values())

Counter({'Europe': 13,
         'Asia': 11,
         'South America': 2,
         'North America': 2,
         'Africa': 2,
         'Oceania': 1})

In [34]:
common_question_train_qa[0]

{'question': 'What country is Rajshahi in?',
 'answer': 'Bangladesh',
 '(city, country)': ['Rajshahi', 'Bangladesh']}

In [36]:
common_question_continent_train_qa = []
tmp_qs = set()
for q in common_question_train_qa:
    country = q['(city, country)'][-1]

    if country in tmp_qs:
        continue
    common_question_continent_train_qa.append(
        {
            "question": f"Which continent is {country} located in?",
            "answer": country2continent[country],
        }
    )
    tmp_qs.add(country)
len(common_question_continent_train_qa)


31

In [37]:
io.dump_jsonlines(common_question_continent_train_qa, f"{vars.DATA_DIR}/debug_meta_train/common_country_data/valid_continent.jsonl")